In [1]:
import pandas as pd
import numpy as np 
import sys
sys.path.append("../tax-calculator")
from taxcalc.records import Records
from taxcalc import *
from taxcalc.utils import *
from numpy.testing import assert_array_almost_equal
import copy

In [2]:
puf = pd.read_csv("../tax-calculator/puf.csv")
policy_base = Policy()
records_base = Records(puf)

policy_T = Policy()
records_reform = Records(puf)

policy_H = Policy()
records_org = Records(puf)
bhv = Behavior()

calcbase = Calculator(policy = policy_base, records = records_base)
calc_T = Calculator(policy = policy_T, records = records_reform)
calc_T_Be = Calculator(policy = policy_H, records = records_org,behavior = bhv)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [3]:
reform_T1 = {
2017: {'_II_rt1': [0.12],
'_II_brk1': [[37500, 75000, 37500, 37500, 75000, 37500]],
'_II_rt2': [0.25],
'_II_brk2': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt3': [0.25],
'_II_brk3': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt4': [0.25],
'_II_brk4': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt5': [0.25],
'_II_brk5': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt6': [0.25],
'_II_brk6': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_II_rt7': [0.33],
'_PT_rt1': [0.12],
'_PT_brk1': [[37500, 75000, 37500, 37500, 75000, 37500]],
'_PT_rt2': [0.25],
'_PT_brk2': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt3': [0.25],
'_PT_brk3': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt4': [0.25],
'_PT_brk4': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt5': [0.25],
'_PT_brk5': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt6': [0.25],
'_PT_brk6': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_PT_rt7': [0.33],
'_CG_thd1': [[37500, 75000, 37500, 37500, 75000, 37500]],
'_CG_thd2': [[112500, 225000, 112500, 112500, 225000, 112500]],
'_AMT_trt1':[0],
'_AMT_trt2':[0],
'_NIIT_trt':[0],
'_STD': [[15000,30000,15000,15000,12600, 6300, 1050]],
'_II_em': [0],
'_ID_BenefitSurtax_crt': [0.0],
'_ID_BenefitSurtax_trt': [1.0],
'_ID_BenefitSurtax_em': [[100000, 200000, 100000, 100000, 200000, 100000]]
}}

reform_be = {2017: {'_BE_sub': [0.55],
                    '_BE_cg': [-3.67]}}

In [4]:
policy_T.implement_reform(reform_T1)
policy_H.implement_reform(reform_T1)
calcbase.advance_to_year(2017)
calc_T.advance_to_year(2017)
calc_T_Be.advance_to_year(2017)

In [5]:
calcbase.calc_all()
calc_T.calc_all()
calc_T_Be.calc_all()
bhv.update_behavior(reform_be)
calc_behav = Behavior.response(calcbase, calc_T_Be)
calc_behav.calc_all()

In [6]:
Trump_static = multiyear_diagnostic_table(calc_T, num_years=10)

In [7]:
Base = multiyear_diagnostic_table(calcbase, num_years=10)

In [8]:
Trump_static.iloc[18] - Base.iloc[18]

2017     -135.6
2018     -140.0
2019     -143.8
2020     -148.4
2021     -154.0
2022     -160.6
2023     -167.5
2024     -175.2
2025     -183.3
2026     -192.4
Name: Combined Liability ($b), dtype: float64

In [9]:
Trump_BE = multiyear_diagnostic_table(calc_behav, num_years=10)

In [10]:
Trump_static

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),172.4,174.1,175.6,177.1,178.4,179.8,181.0,182.3,183.6,184.8
AGI ($b),"10,960.8","11,406.5","11,828.8","12,284.2","12,786.1","13,327.2","13,900.3","14,505.9","15,142.5","15,815.1"
Itemizers (#m),21.1,21.9,22.6,23.4,24.2,25.0,25.8,26.6,27.5,28.3
Itemized Deduction ($b),923.4,987.9,"1,052.6","1,122.6","1,196.6","1,274.5","1,354.9","1,437.2","1,524.7","1,617.8"
Standard Deduction Filers (#m),137.9,138.6,139.0,139.5,139.6,139.9,140.0,140.2,140.4,140.6
Standard Deduction ($b),"2,770.0","2,848.0","2,925.0","3,005.0","3,083.4","3,167.6","3,248.6","3,334.7","3,421.9","3,512.6"
Personal Exemption ($b),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taxable Income ($b),"7,854.2","8,172.0","8,467.4","8,785.1","9,149.9","9,546.5","9,971.8","10,424.3","10,903.0","11,408.8"
Regular Tax ($b),"1,607.3","1,670.4","1,727.5","1,790.6","1,865.1","1,946.7","2,035.9","2,131.4","2,233.4","2,341.8"
AMT Income ($b),"10,453.2","10,863.0","11,249.3","11,666.3","12,126.5","12,623.3","13,151.5","13,709.9","14,296.6","14,915.9"


In [11]:
Trump_BE

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
Returns (#m),172.4,174.1,175.6,177.1,178.4,179.8,181.0,182.3,183.6,184.8
AGI ($b),"11,519.2","11,981.0","12,421.2","12,895.3","13,420.2","13,987.9","14,585.8","15,220.6","15,883.6","16,589.6"
Itemizers (#m),20.7,21.5,22.1,22.9,23.7,24.5,25.4,26.2,27.0,27.9
Itemized Deduction ($b),932.7,998.4,"1,063.8","1,134.8","1,211.6","1,289.6","1,371.0","1,456.6","1,543.9","1,638.4"
Standard Deduction Filers (#m),138.4,139.0,139.5,139.9,140.1,140.4,140.5,140.6,140.8,141.1
Standard Deduction ($b),"2,780.3","2,858.4","2,936.6","3,017.5","3,096.1","3,180.3","3,261.7","3,346.6","3,435.6","3,526.9"
Personal Exemption ($b),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taxable Income ($b),"8,390.2","8,722.9","9,034.5","9,369.2","9,754.3","10,176.7","10,625.3","11,105.0","11,612.7","12,147.5"
Regular Tax ($b),"1,764.9","1,832.3","1,893.9","1,961.9","2,042.4","2,131.4","2,227.6","2,331.2","2,441.7","2,558.6"
AMT Income ($b),"10,989.7","11,413.6","11,816.2","12,249.9","12,729.9","13,252.9","13,803.7","14,388.7","15,000.2","15,650.8"
